In [1]:
import random
import re, string,os
from glob import glob as gb
import pandas as pd
from collections import Counter
from tqdm import tqdm
from datetime import datetime, timedelta, date
from collections import OrderedDict
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import math
from utils import frequencyparser
from utils import pmi
from utils import utils
from utils import cluster

base_path = "/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/ParlaMintCase"

In [2]:
df = utils.import_all('pl',format='lemmatized')
months = utils.month_generator("2019-09","2020-08")
stopwords = set(pd.read_csv(os.path.join(base_path,"resources","stopwords","pl.csv")).iloc[:,0])

In [3]:
d = pd.DataFrame()

for m in months:
    rdf = pmi.get_pmi_table(df = df,date = m, window = 24, word = "polityka", stopwords = stopwords)
    rdf = rdf.sort_values('npmi').reset_index(drop=True).iloc[:50,:]
    rdf['date'] = m
    d = d.append(rdf)

In [4]:
candidate_words = [list(d[d['date'] == y].sort_values('npmi')['w']) for y in list(set(d['date']))]
candidate_words = set([item for sublist in candidate_words for item in sublist if item])

sim_matrix = cluster.generate_matrix(list_words=candidate_words,model_name=base_path+"/resources/embeddings/word2vec_100_3_polish.bin")

created list with 203 words


In [6]:
clustered = cluster.cluster_word(sim_matrix,12)
colors = {c:"#{:06x}".format(random.randint(0, 0xFFFFFF)) for w,c in clustered.items()}

In [7]:
set_words = set()
dr = pd.DataFrame()
for y in sorted(list(set(d['date']))):
    list_words = list(d[d['date'] == y].sort_values('npmi')['w'])
    if len(list_words) < 50:
        list_words = list_words + ["" for x in range(50 - len(list_words))]
    else:
        list_words = list_words[:50]
    set_words = set_words | set(list_words)
    dr[y] = list_words
str_html = dr.to_html()
for w in set_words:
    try:
        str_html = str_html.replace(f"<td>{w}",f'<td style="background-color: {colors[clustered[w]]}">{w}')
    except:
        continue
with open(os.path.join(base_path,"results","res.html"),'w') as f:
    f.write(str_html)
